# Window Functions, Part 3: Analytic Functions

We're going to introduce a few widely used analytic functions that are often used with window functions. Thanks to them, you'll be able to write queries that are useful when making business decisions. 


Checkout the fiddle here:https://www.db-fiddle.com/f/vYW8BSKZHVwKzbgXd8GY5u/3

Now, let's explore the tables!

# The website table

Select all the information from the website table.

Each website has an id, name, monthly budget and the date of opening (opened).

# The statistics table

Select all the information from the statistics table.

In this table, you can find the statistics from May 2016. Each generated statistics row corresponds to a certain website_id and a particular day. Column users shows how many unique people visited the website that day. Column impressions indicates how many times an advertisement was shown to a user. clicks means the number of clicks on an advertisement. Column revenue is the total profit generated by those clicks on that day.


# General Syntax

Let's introduce the general syntax for analytic functions:

```
<analytic function> OVER (...)
```

Looks familiar? That's right. 

Analytic functions are quite easy, their syntax is identical to aggregate functions. 

The difference is that aggregate functions calculate a cumulative result for all the rows in the window frame, while the analytic functions refer to single rows within the frame.


# Function LEAD(x)

Alright. Take a look at the very first example:

```
SELECT
  name, 
  opened,
  LEAD(name) OVER(ORDER BY opened)
FROM website;
```


The analytic function here is `LEAD(name)`.

`LEAD` with a single argument in the parentheses looks at the next row in the given order and shows the value in the column specified as the argument. 

Take a look:

<img src = 'https://learnsql.com/static/window-functions-window-functions-part6-ex5.png'>

Exercise
Run the template from the instruction and study the result of LEAD(name).

Note that for each website, the function shows the name of the website that was opened next. The last row doesn't show any value, because there is no 'next' row.


# Exercise


Run the template from the instruction and study the result of LEAD(name).

Note that for each website, the function shows the name of the website that was opened next. The last row doesn't show any value, because there is no 'next' row.



```
SELECT
  name,
  opened,
  LEAD(name) OVER(ORDER BY opened)
FROM website;
```

Note that the column specified as the argument of LEAD may be different than the column by which we sort rows:

```
SELECT
  name,
  opened,
  budget,
  LEAD(budget) OVER(ORDER BY opened)
FROM website;
```

The above query will still sort the rows by the opening date, but will show the budget of the next website instead of its opening date.

Now it's your turn to write a query with LEAD(x).

## Exercise

For all the statistics of the website with id = 1, show the day, the number of users and the number of users on the next day.

```
SELECT
  day,
  users,
  LEAD(users) OVER(ORDER BY day)
FROM statistics
WHERE website_id = 1;
```

# Deltas

LEAD can be extremely useful when we want to calculate deltas, i.e. differences between two values. A typical example may look like this:

```
SELECT
  day,
  clicks,
  LEAD(clicks) OVER(ORDER BY day),
  clicks - LEAD(clicks) OVER(ORDER BY day) 
FROM statistics
WHERE website_id = 2;
```

The above query calculates day-to-day deltas: the last column shows the difference in clicks between the current day and the next day. 

From a business point of view, this could easily tell us a lot about the website: if the deltas for many rows are positive, and possibly increasing, then the website is expanding. 

If, in turn, the deltas are mostly negative, we can start to worry about the performance of the website.

# Exercise
For website_id = 1, show each statistics row: day, revenue, revenue on the next day and the difference between these two values.


```
SELECT
  day,
  revenue,
  LEAD(revenue) OVER(ORDER BY day),
  revenue - LEAD(revenue) OVER(ORDER BY day)
FROM statistics
WHERE website_id = 1;

```

# Function LEAD(x, y)

There is also another version of LEAD. It takes two arguments: LEAD(x,y). x remains the same – it specifies the column to return. y, in turn, is a number which defines the number of rows forward from the current value. 

For instance:

```
SELECT
  name,
  opened,
  LEAD(opened,2) OVER(ORDER BY opened)
FROM website;
```

This form of LEAD won't show the webpage with the opening date coming immediately after the current opening date. 

Instead, it will show the opening date 2 rows forward – the 1st row will show the 3rd date, etc.

# Exercise

Take the statistics for the website with id = 2 between 1 and 14 May 2016 and show the day, the number of users and the number of users 7 days later.

Note that the last 7 rows don't have a value in the last column, because no rows '7 days from now' can be found for them.



```
SELECT
  day,
  users,
  LEAD(users, 7) OVER(ORDER BY day)
FROM statistics
WHERE website_id = 2
  AND day BETWEEN '2016-05-01' AND '2016-05-14';
 ```

# Function LEAD(x, y, z)

The last possible type of LEAD takes three arguments:

```
SELECT
  name,
  opened,
  LEAD(opened,2,'2000-01-01') OVER(ORDER BY opened)
FROM website;
```

The new (last) argument tells the function what it should return if no matching value is found. 

Previously, the last rows got NULLs from the function, because there were no "lead" (further) rows for them. 

Now, you can specify what should be displayed in such cases instead of the default NULL. Here, we show '2000-01-01'. 

Note that this value must be of the same type as the column itself: if you show dates with LEAD, the last argument must be a date too. 

You can't show "not available" or 0 instead.

# Exercise

Modify the template based on the previous exercise so that it shows -1 instead of NULL if no LEAD value is found.



```
SELECT
  day,
  users,
  LEAD(users, 7, -1) OVER(ORDER BY day)
FROM statistics
WHERE website_id = 2
  AND day BETWEEN '2016-05-01' AND '2016-05-14';
```

# Function LAG(x)

Of course, there's also a function that shows a previous value, and its name is LAG(x):

```
SELECT
  name,
  opened,
  LAG(name) OVER(ORDER BY opened)
FROM website;
```

Now, instead of showing the next opening date, we show the previous opening date. 

Take a look:

<img src = 'https://learnsql.com/static/window-functions-window-functions-part6-ex10.png'>


Note that you can always sort the rows in the reverse order with DESC and use LEAD(...) instead of LAG(...), or the other way around. 

In other words:

```
LEAD (...) OVER(ORDER BY ...)
```

is the same as

```
LAG (...) OVER (ORDER BY ... DESC)
```

and

```
LEAD (...) OVER(ORDER BY ... DESC)
```

is the same as

```
LAG (...) OVER (ORDER BY ...)
```

# Exercise

Show the statistics for the website with id = 3: day, number of clicks that day and the number of clicks on the previous day.

Note that there won't be any previous value for the first row.



```
SELECT
  day,
  clicks,
  LAG(clicks) OVER(ORDER BY day)
FROM statistics
WHERE website_id = 3;
```

# Function LAG(x, y)

And, of course, there's also an analogous version of LAG(x,y) with two arguments:

```
SELECT
  name,
  opened,
  LAG(opened,2) OVER(ORDER BY opened)
FROM website;
```

Now, instead of the previous row, we show the value '2 rows before'.

Some people have difficulties remembering the difference between `LEAD(...)` and `LAG(...)`. 

You may think about it in the following way: let's imagine that your ordered rows are people hiking in the mountains. 

The people (rows) that are FURTHER ahead are currently LEADing, so you get them with LEAD. 

The people (rows) BEHIND you are currently LAGging (making progress slower), so you call them with LAG.

# Exercise

Show the statistics for the website with id = 3: day, revenue and the revenue 3 days before.

```
SELECT
  day,
  revenue,
  LAG(revenue, 3) OVER(ORDER BY day)
FROM statistics
WHERE website_id = 3;
```

# Function LAG(x, y, z)

The last version of LAG takes three arguments and is very analogous to LEAD with three arguments:

```
SELECT
  name,
  opened,
  LAG(opened,2,'2000-01-01') OVER(ORDER BY opened)
FROM website;
```

The last argument is the value displayed when no 'lag' (previous) row can be found. 

Remember that this value must be of the type corresponding to the column.

# Exercise

Modify the template from the previous exercise so that it shows -1.00 for rows with no revenue value 3 days before.


```
SELECT
  day,
  revenue,
  LAG(revenue, 3, -1.00) OVER(ORDER BY day)
FROM statistics
WHERE website_id = 3;
```

# FIRST_VALUE(x)

LEAD and LAG are 2 functions which are always relative to the current row. 

Now, we'll get to know three other functions that are independent of the current row.

The first one is `FIRST_VALUE(x)`. As you may guess, it returns the first value in the column x in the given order. 

Take a look:

```
SELECT
  name,
  opened,
  budget,
  FIRST_VALUE(budget) OVER(ORDER BY opened)
FROM website;
```

Here, we still sort rows by the opening date (`ORDER BY opened`), but we show the budget for the earliest opening date (`FIRST_VALUE(budget)`). 

In this way, we can show the budget for the website that was opened first.

Note that this would be impossible to achieve with a simple `MIN(...)` function. `MIN(budget)` would simply show the smallest budget: 500 in this case. 

That is not the same as the budget of the first website (3000).

# Exercise

Show the statistics for website_id = 2. For each row, show the day, the number of users and the smallest number of users ever.


```
SELECT
  day,
  users,
  FIRST_VALUE(users) OVER(ORDER BY users)
FROM statistics
WHERE website_id = 2;
```

# Exercise

Show the statistics for website_id = 3. For each row, show the day, the revenue and the revenue on the first day.

```
SELECT
  day,
  revenue,
  FIRST_VALUE(revenue) OVER(ORDER BY day)
FROM statistics
WHERE website_id = 3;
```

# LAST_VALUE(x)


Of course, we can also find the last value: simply use LAST_VALUE(x) instead:

```
SELECT
  name,
  opened,
  LAST_VALUE(opened) OVER(ORDER BY opened)
FROM website;
```

Now, we'll see the opening date of each website plus the opening day of the last (most recent) website... or will we? 

Let's find out.

As you can see, the result is a bit different from our expectations. LAST_VALUE shows the current value instead of the highest value.

# LAST_VALUE(x) with window frame


Well... The query didn't work! 

Instead of the latest opening date, we saw the current opening date.

In order to understand why this happened, we need to refer to the previous part of our course, where we talked about default window frames:

If there is an `ORDER BY` clause, `RANGE UNBOUNDED PRECEDING` will be used as the default window frame.

And this is precisely the cause of our troubles. 

We indeed used `ORDER BY` within `OVER(...)`, which is why `LAST_VALUE(x)` only considers the rows from the first row until the current row. 

The solution is quite simple: we need to define the right window frame:

```
SELECT
  name,
  opened,
  LAST_VALUE(opened) OVER(
    ORDER BY opened
    ROWS BETWEEN UNBOUNDED PRECEDING
      AND UNBOUNDED FOLLOWING)
FROM website;
```


# LAST_VALUE(x) – practice

The query worked fine this time. Here's one thing you should remember: while FIRST_VALUE works well with the default window frame, LAST_VALUE needs an explicit definition of the right window frame to actually make sense.

Of course, you can order by one column and return the other:

```
SELECT
  name,
  budget,
  LAST_VALUE(budget) OVER(
    ORDER BY opened
    ROWS BETWEEN UNBOUNDED PRECEDING
      AND UNBOUNDED FOLLOWING)
FROM website;
```

Here we show the budget of the most recent website (not the greatest budget).

# Exercise

Show the statistics for website_id = 1. For each row, show the day, the number of impressions and the number of impressions on the day with the most users.

```
SELECT
  day,
  impressions,
  LAST_VALUE(impressions) OVER(
    ORDER BY users
    ROWS BETWEEN UNBOUNDED PRECEDING
      AND UNBOUNDED FOLLOWING)
FROM statistics
WHERE website_id = 1;
```

# NTH_VALUE(x, n)

The last function we'll learn in this part is: `NTH_VALUE(x,n`). This function returns the value in the column x of the nth row in the given order.

```
SELECT
  name,
  opened,
  NTH_VALUE(opened, 2) OVER(
    ORDER BY opened
    ROWS BETWEEN UNBOUNDED PRECEDING
      AND UNBOUNDED FOLLOWING)
FROM website;
```

This time, we're showing the opening date of the current row together with the second row when sorted by the opening date. With `NTH_VALUE`, we also need to redefine the window frame. Otherwise, some rows will display incorrect values.

Just to remind you, you can always revert the order by adding the word DESC, which sometimes comes in handy with NTH_VALUE:

`...OVER(ORDER BY opened DESC)...`

# Exercise

Take the statistics for the website with id = 2 between May 15 and May 31, 2016. Show the day, the revenue on that day and the third highest revenue in that period.

```
SELECT
  day,
  revenue,
  NTH_VALUE(revenue,3) OVER (
    ORDER BY revenue DESC
    ROWS BETWEEN UNBOUNDED PRECEDING
      AND UNBOUNDED FOLLOWING)
FROM statistics
WHERE website_id = 2
  AND day BETWEEN '2016-05-15' AND '2016-05-31';
```

# Exercise

Take the statistics for website_id = 1. For each row, show the day, the number of clicks on that day and the median of clicks in May 2016 (calculated as the 16th value of all 31 values in the column clicks when sorted by the number of clicks).



```
SELECT
  day,
  clicks,
  NTH_VALUE(clicks, 16) OVER(
    ORDER BY clicks DESC
    ROWS BETWEEN UNBOUNDED PRECEDING
      AND UNBOUNDED FOLLOWING)
FROM statistics
WHERE website_id = 1;
```

# Exercise

For each statistics row of website_id = 3, show the day, the number of clicks on that day and a ratio expressed as percentage: the number of clicks on that day to the greatest number of clicks on any day. Round the percentage to integer values.



```
SELECT
  day,
  clicks,
  ROUND(CAST(clicks AS numeric) / LAST_VALUE(clicks) OVER(
    ORDER BY clicks
    ROWS BETWEEN UNBOUNDED PRECEDING
      AND UNBOUNDED FOLLOWING) * 100)
FROM statistics
WHERE website_id = 3;
```


# Summary

That's all I wanted to teach you today. Let's review what we've learned:

- `LEAD(x)` and `LAG(x)` give you the next/previous value in the column x, respectively.

- `LEAD(x,y)` and `LAG(x,y)` give you the value in the column x of the row which is y rows after/before the current row, respectively.

- `FIRST_VALUE(x)` and `LAST_VALUE(x)` give you the first and last value in the column x, respectively.

- `NTH_VALUE(x,n)` gives you the value in the column x of the n-th row.

- `LAST_VALUE` and `NTH_VALUE` usually require the window frame to be set to `ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING`.


```
```

# Homework

Use the `statistics` and `website` tables from the beginning of class for the following problems.

https://www.db-fiddle.com/f/vYW8BSKZHVwKzbgXd8GY5u/3# Exercise

RPM (revenue per thousand impressions) is defined as the revenue divided by the number of impressions, times 1000.

RPM = (revenue / number of impressions) * 1000
For each statistics row with website_id = 2, show the day, the RPM and the RPM 7 days later. 

Rename the columns to RPM and RPM_7.

```
SELECT
  day,
  CAST(revenue AS numeric) / impressions * 1000 AS RPM,
  CAST(LEAD(revenue, 7) OVER(ORDER BY day) AS numeric) / LEAD(impressions, 7) OVER(ORDER BY day) * 1000 AS RPM_7
FROM statistics
WHERE website_id = 2;
```

# Exercise

Let's define conversion rate as the number of clicks divided by the number of impressions, multiplied by 100 to obtain percentages.

For website_id = 1 and dates between May 15 and May 31, show each statistics row: day, clicks, impressions, conversion rate (as the conversion column) and the conversion rate on the previous day (as the previous_conversion column).



```
SELECT
  day,
  clicks,
  impressions,
  CAST(clicks AS numeric) / impressions * 100 AS conversion,
  CAST(LAG(clicks) OVER(ORDER BY day) AS numeric) / LAG(impressions) OVER(ORDER BY day) * 100 AS previous_conversion
FROM statistics
WHERE website_id = 1
  AND day BETWEEN '2016-05-15' AND '2016-05-31';
```

# Exercise

For each statistics rows with website_id = 1, show the day, the number of users, the number of users on the last day and the difference between these two values.

```
SELECT
  day,
  users,
  LAST_VALUE(users) OVER(
    ORDER BY day
    ROWS BETWEEN UNBOUNDED PRECEDING
      AND UNBOUNDED FOLLOWING),
  users - LAST_VALUE(users) OVER(
    ORDER BY day
    ROWS BETWEEN UNBOUNDED PRECEDING
      AND UNBOUNDED FOLLOWING)
FROM statistics
WHERE website_id = 1;
```

# Exercise

Let's run some cross-website statistics now. Take the day May 14, 2016 and for each row, show: website_id, revenue on that day, the highest revenue from any website on that day (AS highest_revenue and the lowest revenue from any website on that day (as lowest_revenue).

```
SELECT
  website_id,
  revenue,
  FIRST_VALUE(revenue) OVER(ORDER BY revenue) AS lowest_revenue,
  LAST_VALUE(revenue) OVER(
    ORDER BY revenue
    ROWS BETWEEN UNBOUNDED PRECEDING
      AND UNBOUNDED FOLLOWING) AS highest_revenue
FROM statistics
WHERE day = '2016-05-14';
```

Use the `advertisement` table, which is in the same fiddle link provided for the next problems

# Exercise

For each row, show the day, the price on that day, the price 7 days earlier and the difference between these two values.

```
SELECT
  day,
  price,
  LAG(price, 7) OVER(ORDER BY day),
  price - LAG(price,7) OVER(ORDER BY day)
FROM advertisement;
```

# Exercise

For each row, show the day, the price on that day, the highest price ever (column name highest_price) and the lowest price ever (column name lowest_price).

```
SELECT
  day,
  price,
  FIRST_VALUE(price) OVER(ORDER BY price) AS lowest_price,
  LAST_VALUE(price) OVER(
    ORDER BY price
    ROWS BETWEEN UNBOUNDED PRECEDING
      AND UNBOUNDED FOLLOWING) AS highest_price
FROM advertisement;
```